In [16]:
# Tratamiento de datos
import numpy as np
import pandas as pd

# Preprocesado y modelado
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle

#Configuración warnings
import warnings
warnings.filterwarnings('once')

In [2]:
df_total = pd.read_pickle('../../../datos_finales/archivo_total_est_enc2.pkl')
df_total.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,total_usuarios
0,1.0,1.0,1.0,1.04,1.00,2.23,-0.486274,0.855582,-0.208909,985
1,1.0,1.0,1.0,1.01,1.05,2.23,-0.425690,0.332722,0.687550,801


In [3]:
lista_cols = ['dias_laborables', 'tiempo']

In [4]:
# Decidimos quitar las columnas porque tenemos overfitting.
df_total.drop(lista_cols, axis = 1, inplace = True)

In [5]:
X = df_total.drop("total_usuarios", axis = 1)
y = df_total["total_usuarios"]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
param = {"max_depth": [4,6,8], 
        "max_features": [1,2,3],
        'subsample'     : [0.5, 1], # parametros que se usan de forma estandar
        'learning_rate' : [0.001, 0.01, 0.1]} # parametros que se usan de forma estandar

In [8]:
gs_gb = GridSearchCV(
            estimator=GradientBoostingRegressor(), 
            param_grid= param, 
            cv=10, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [9]:
gs_gb.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [4, 6, 8], 'max_features': [1, 2, 3],
                         'subsample': [0.5, 1]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [10]:
boosting = gs_gb.best_estimator_
boosting

GradientBoostingRegressor(max_depth=4, max_features=3, subsample=0.5)

In [11]:
y_pred_test_gb = boosting.predict(x_test)
y_pred_train_gb = boosting.predict(x_train)

In [12]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    """
    Esta función nos dice las métricas de nuestro modelo.
    Args:
        y_test (Serie): el test de nuestra variable respuesta.
        y_train (Serie): el train de nuestra variable respuesta.
        y_test_pred (Serie): la predicción para el test de nuestra variable respuesta.
        y_train_pred (Serie): la predicción para el train de nuestra variable respuesta.
        tipo_modelo (str): nombre que le damos al modelo.
    Returns:
        dataframe con todas las métricas de nuestro modelo.
    """
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [13]:
dt_results = metricas(y_test, y_train, y_pred_test_gb, y_pred_train_gb, "Gradient Boosting")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,436.401188,324922.558029,570.019787,0.905072,test,Gradient Boosting
1,286.234247,137809.693621,371.227280,0.963883,train,Gradient Boosting


In [14]:
dt_results.to_pickle('../../../datos_finales/predicciones/metricas_totales.pkl')

Este modelo tiene una métrica muy buena para R2. No obstante, existe overfitting en este modelo aunque es mejor que en el decision tree, el random forest, o el xg boost. Por tanto, decidimos quedarnos con este modelo para usuarios totales.

In [15]:

importancia_predictores = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': boosting.feature_importances_}
                            )


importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
4,temperatura,0.306993
1,año,0.257148
0,estacion,0.114910
2,mes,0.111194
5,humedad,0.110964
6,velocidad_viento,0.081836
3,dia_semana,0.016956


In [17]:
with open ("../../../datos_finales/predicciones/mejor_modelo_total.pkl", "wb") as f:
    pickle.dump(boosting, f)